In [43]:
import pandas as pd

# Load data (ensure to replace the path securely or use environment variables)
#data = pd.read_csv(r'C:\Users\jpscw\Documents\EDA Anahy\full_data_best_secret.csv')

# Prompt the user to input the file path for the CSV file
#file_path = input("Please enter the path to the CSV file: ")

# Step 2: Load the CSV into a DataFrame
#try:
   # data = pd.read_csv(file_path)
   # print("CSV file loaded successfully.")
#except Exception as e:
   # print(f"Error loading CSV file: {e}")
   # exit()  # Exit if the file cannot be loaded


Filter rows where qty_items_sold == 1, then group by order_code.

In [44]:
# Filter the data to keep only rows where qty_items_sold == 1
df_filtered = data[data['qty_items_sold'] == 1]

# Group by 'order_code' and filter out groups with more than 1 row (i.e., keep single-item orders)
df_single = df_filtered.groupby('order_code').filter(lambda x: len(x) == 1)

# Count total products for each category (using value_counts)
total_counts = df_single['product_navision_detail_category'].value_counts().reset_index()
total_counts.columns = ['product_navision_detail_category', 'total_count']

# Count occurrences of each box type for each category
box_counts = df_single.groupby(['product_navision_detail_category', 'display_name']).size().reset_index(name='count')

# Merge counts to calculate percentages
merged = pd.merge(box_counts, total_counts, on='product_navision_detail_category')
merged['percentage'] = (merged['count'] / merged['total_count']) * 100

# Pivot the results to get the percentage table for box types across categories
box_distribution_percentage = merged.pivot_table(
    index='product_navision_detail_category',
    columns='display_name',
    values='percentage',
    fill_value=0
)


Identify categories with any box type used more than 90%.

In [45]:
# Filter for categories with a box type used > 90%
categories_with_high_usage = merged[merged['percentage'] >= 90]

# Get the unique categories that meet the condition
unique_categories = categories_with_high_usage['product_navision_detail_category'].unique()

# Count unique categories with box usage > 90%
number_unique_categories = len(unique_categories)


We create a function to calculate the volume of each box type and return a sorted table.

In [46]:
def calculate_box_volumes(df):
    df['width_in_millimeter'] = pd.to_numeric(df['width_in_millimeter'], errors='coerce')
    df['length_in_millimeter'] = pd.to_numeric(df['length_in_millimeter'], errors='coerce')
    df['height_in_millimeter'] = pd.to_numeric(df['height_in_millimeter'], errors='coerce')

    # Calculate the volume for each row
    df['volume_in_cubic_mm'] = df['width_in_millimeter'] * df['length_in_millimeter'] * df['height_in_millimeter']
    
    # Group by 'display_name' and calculate average volume for each box type
    box_volumes = df.groupby('display_name')['volume_in_cubic_mm'].mean().reset_index()
    
    # convert from cubic millimeters to cubic centimeters for more intuitive results
    box_volumes['volume_in_cubic_cm'] = box_volumes['volume_in_cubic_mm'] / 1000  # 1 cm³ = 1000 mm³

    # Sort the results for easier interpretation
    box_volumes_sorted = box_volumes.sort_values(by='volume_in_cubic_cm', ascending=False).reset_index(drop=True)
    
    return box_volumes_sorted

# Usage
box_volumes_single = calculate_box_volumes(df_single)



We will identify the most used box types for each category and merge their volumes with the categories.

In [47]:
# Identify the most used box for each category
most_used_boxes = box_distribution_percentage.idxmax(axis=1)
most_used_percentages = box_distribution_percentage.max(axis=1)

# Create a DataFrame for the categories with box usage > 90%
high_usage_categories = most_used_boxes[most_used_percentages >= 90].reset_index()
high_usage_categories.columns = ['product_navision_detail_category', 'most_used_box']

# Merge to get the volume of the most used box
high_usage_categories = pd.merge(
    high_usage_categories,
    box_volumes_single[['display_name', 'volume_in_cubic_cm']],
    left_on='most_used_box',
    right_on='display_name',
    how='left'
)

# Keep only the relevant columns
high_usage_categories = high_usage_categories[['product_navision_detail_category', 'volume_in_cubic_cm']]


Now we apply a function to calculate weighted volumes based on usage percentages.

In [48]:
def calculate_weighted_volume(row, box_volumes):
    used_boxes = row[row > 0]  # Select boxes with non-zero usage
    box_names = used_boxes.index.tolist()  # List of box types used for this category

    # Get corresponding volumes for these boxes
    used_box_volumes = box_volumes[box_volumes['display_name'].isin(box_names)]
    
    weighted_sum = 0
    for box in used_box_volumes['display_name']:
        percentage = row[box] / 100  # Convert percentage to decimal
        volume = used_box_volumes[used_box_volumes['display_name'] == box]['volume_in_cubic_cm'].values[0]
        weighted_sum += percentage * volume
    
    num_boxes_used = len(used_boxes)
    weighted_volume = weighted_sum / num_boxes_used if num_boxes_used > 0 else 0
    return weighted_volume

# Calculate the weighted volume for each category
box_volumes_mapping = box_volumes_single[['display_name', 'volume_in_cubic_cm']]
box_distribution_percentage['weighted_volume'] = box_distribution_percentage.apply(calculate_weighted_volume, axis=1, box_volumes=box_volumes_mapping)


In [49]:
# Create a dictionary for high usage categories with the most used box volume
high_usage_volume_dict = high_usage_categories.set_index('product_navision_detail_category')['volume_in_cubic_cm'].to_dict()

# Create a dictionary for weighted volumes
weighted_volume_dict = box_distribution_percentage['weighted_volume'].to_dict()

# Define a function to assign the appropriate volume based on category
def assign_approximated_volume(row):
    category = row['product_navision_detail_category']
    
    if category in high_usage_volume_dict:
        return high_usage_volume_dict[category]
    elif category in weighted_volume_dict:
        return weighted_volume_dict[category]
    else:
        return 0

# Apply the function to create the 'approximated_volume' column in df_single
df_single['approximated_volume'] = df_single.apply(assign_approximated_volume, axis=1)

# Create a DataFrame with unique product categories and their approximated volumes
df_single_unique = df_single[['product_navision_detail_category', 'approximated_volume']].drop_duplicates(subset='product_navision_detail_category')


Now we get the approximated volumes for the rest of categories (that have no errors in the columns 'qty_items_sold', 'qty_item')

In [50]:
# Step 1: Clean the orders where 'qty_items_sold' doesn't match the 'qty_items'
# Group by 'order_code' and sum 'qty_items_sold'
df_qty_items_sold_sum = data.groupby('order_code')['qty_items_sold'].sum().reset_index()

# Group by 'order_code' and take the first value of 'qty_items'
df_qty_items_first = data.groupby('order_code')['qty_items'].first().reset_index()

# Merge the two dataframes on 'order_code' for comparison
df_comparison = pd.merge(df_qty_items_sold_sum, df_qty_items_first, on='order_code', suffixes=('_sold_sum', '_first'))

# Create a new column 'are_same' to compare the two columns and filter mismatches
df_comparison['are_same'] = df_comparison['qty_items_sold'] == df_comparison['qty_items']

# Filter rows where the values are not the same
df_not_same = df_comparison[df_comparison['are_same'] == False]

# Extract order codes with mismatches
order_codes_to_remove = df_not_same['order_code']

# Remove rows from the original data where order codes have mismatches
df_cleaned = data[~data['order_code'].isin(order_codes_to_remove)]

# Step 2: Identify categories that are in 'df_cleaned' but not in 'df_single_unique'
categories_df_single = set(df_single_unique['product_navision_detail_category'])
categories_df_cleaned = set(df_cleaned['product_navision_detail_category'])

# Find categories not in df_single
categories_not_in_single = categories_df_cleaned - categories_df_single
filtered_data = df_cleaned[df_cleaned['product_navision_detail_category'].isin(categories_not_in_single)]

# Step 3: Process orders with only one unique category from the excluded ones
orders_with_single_excluded_category = filtered_data.groupby('order_code').filter(
    lambda x: x['product_navision_detail_category'].nunique() == 1
)

# Count the occurrences of each category in the filtered orders
category_counts = orders_with_single_excluded_category['product_navision_detail_category'].value_counts()

# Step 4: Create a dictionary for each order_code with associated categories
grouped_categories = filtered_data.groupby('order_code')['product_navision_detail_category'].agg(list)
categories_dict = grouped_categories.to_dict()

# Step 5: Calculate maximum volume for excluded categories
max_volume_results = {}

"""
This function iterates over each order and calculates the approximate volume of products
that belong to excluded categories (categories not present in df_single_unique). It works by:
1. Determining the box volume used for each order.
2. Calculating the total adjusted volume for included categories based on the quantity of items sold.
3. Estimating the remaining volume in the box for excluded categories.
4. Tracking the maximum estimated volume for each excluded category across all orders.
The results are stored in the `max_volume_results` dictionary.
"""

# Iterate over each order code and its associated categories
for order_code, categories in categories_dict.items():
    # Retrieve box volume used for the order
    box_used = data[data['order_code'] == order_code]['display_name'].unique()

    if len(box_used) == 0:
        continue  # Skip if no box information is found

    box_display_name = box_used[0]

    # Retrieve the box volume
    box_volume_row = box_volumes_single[box_volumes_single['display_name'] == box_display_name]

    if box_volume_row.empty:
        continue  # Skip if no box volume is found

    box_volume_cm3 = box_volume_row['volume_in_cubic_cm'].values[0]

    # Filter out the categories that are in df_single_unique and calculate adjusted volumes
    approximated_volumes = df_single_unique[df_single_unique['product_navision_detail_category'].isin(categories)]
    
    # Merge with qty_items_sold to calculate adjusted volume
    merged_data = pd.merge(approximated_volumes, 
                           data[data['order_code'] == order_code][['product_navision_detail_category', 'qty_items_sold']],
                           on='product_navision_detail_category', 
                           how='left')

    merged_data['adjusted_volume'] = merged_data['approximated_volume'] * merged_data['qty_items_sold']
    total_approximated_volume = merged_data['adjusted_volume'].sum()

    # Calculate approximate volume for excluded category
    approximate_volume_excluded_category = box_volume_cm3 - total_approximated_volume

    # Track the maximum volume for the excluded category
    excluded_categories = set(categories) - set(df_single_unique['product_navision_detail_category'])

    if excluded_categories:
        excluded_category = excluded_categories.pop()

        if excluded_category in max_volume_results:
            max_volume_results[excluded_category] = max(max_volume_results[excluded_category], approximate_volume_excluded_category)
        else:
            max_volume_results[excluded_category] = approximate_volume_excluded_category

# Step 6: Convert the results to DataFrame
results_df = pd.DataFrame(list(max_volume_results.items()), columns=['category', 'approximate_volume'])

# Step 7: Merge results with the unique category volumes
results_df = results_df.rename(columns={'category': 'product_navision_detail_category', 'approximate_volume': 'approximated_volume'})

# Combine with existing approximated volumes DataFrame
combined_df_final = pd.concat([df_single_unique, results_df], ignore_index=True)

# Filter out entries with negative or zero volumes
combined_df_final = combined_df_final[combined_df_final['approximated_volume'] > 0]

# Step 4: Create the final dictionary of volumes
final_volume_dict = combined_df_final.set_index('product_navision_detail_category')['approximated_volume'].to_dict()


In [51]:
###IN CASE YOU PREFER TO WRITE WITH A JSON FILE
#import json
#with open('volume_dict.json', 'w') as f:
    #json.dump(final_volume_dict, f)

In [60]:
# Write the dictionary to a .py file with UTF-8 encoding
with open('volume_dict.py', 'w', encoding='utf-8') as f:
    f.write('volume_dict = ' + str(final_volume_dict))